In [ ]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
import os
from langchain.agents import Tool, initialize_agent
from langchain.tools import StructuredTool
from typing import Annotated
from langchain_core.documents import Document
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.agents.agent_types import AgentType
from pydantic import BaseModel, Field


In [43]:
from retriever import load_hybrid_retriever

index_path = "../vectorstore/faiss_vectorestore"
pickle_path = "../vectorstore/documents.pkl"
model_path = "../multilingual-e5-large"

hybrid_retriever = load_hybrid_retriever(index_path, pickle_path, model_path)


In [44]:
retrieved_docs = hybrid_retriever.get_relevant_documents("Identify the members of the board of directors in 2021.")
retrieved_docs

[Document(id='e12830ca-597d-4746-ae67-e04f817227be', metadata={'source': 'Renault URD 2023.pdf', 'id': '49596059-8681-4855-b102-7897c6d05aee'}, page_content='(2) Excluding the directors representing employees and the director representing employee shareholders.\nRenault Group I Universal registration document 2023 Strategy and Sustainability Committee 6 members 50% Independent (2) www.renaultgroup.com Renault Group Leadership Team Presentation of the Leadership Team as of March 1, 2024'),
 Document(id='17a756dc-9e39-4683-9017-156417f74209', metadata={'source': 'Renault URD 2023.pdf', 'id': '6bf00ebc-b456-4136-8a98-1e5649e66bf3'}, page_content='Financial statements Information about the company, the capital and the share ownership\nAnnual general meeting Additional information Annual general meeting Fourteenth resolution (Approval of the components of the overall compensation and benefits of any kind paid during or awarded for the financial year ended December 31, 2023 to Mr. Jean-Domin

In [ ]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

# Configuration du modèle
llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4o", temperature=0.1)

# Supposons que vous avez déjà votre retriever configuré
# retriever = votre_vectorstore.as_retriever(search_kwargs={"k": 5})

# Création de la chaîne QA
prompt_template = """
Utilisez les éléments de contexte suivants pour répondre à la question.
Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas.

Contexte: {context}

Question: {question}

Réponse utile:
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

rag_qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever, 
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# Fonction pour rechercher dans la base de connaissances
def retriever_relevant_docs(query):
    result = rag_qa_chain.invoke({"query": query})
    return result["result"]

# Création de l'outil de recherche
rag_tool = Tool(
    name="rag",
    func=retriever_relevant_docs,
    description="Recherche dans la base de connaissances pour répondre aux questions basées sur les documents indexés"
)

# Mémoire pour maintenir le contexte de conversation
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Initialisation de l'agent avec gestion d'erreurs et mémoire
agent = initialize_agent(
    tools=[rag_tool],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=False,
    handle_parsing_errors=True
)

# Utilisation de l'agent
def ask_agent(question):
    return agent.run(question)

# Exemple d'utilisation
if __name__ == "__main__":
    # Posez vos questions
    response = ask_agent("Identify the members of the board of directors in 2021 at Renault")
    print(response)

To find the members of the board of directors at Renault in 2021, I recommend checking Renault's official website or their annual reports for that year. These documents typically include detailed information about the board's composition. Alternatively, you might find this information in Renault's 2021 Universal Registration Document, which is often available on their investor relations page.


In [5]:
# Exemple d'utilisation
if __name__ == "__main__":
    # Posez vos questions
    response = ask_agent("Quelle est la question que vous voulez poser ?")
    print(response)

C:\Users\habou\AppData\Local\Temp\ipykernel_2900\2855847617.py:70: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return agent.run(question)




> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I'm here to assist you with any questions you might have. Please feel free to ask anything, and I'll do my best to provide a helpful response.`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 